# Zajęcia 1 (część 1)

Zakres:
* Zapoznanie sie ze srodowiskiem pracy (GKE na GCP)
* Operacje na obiektowym systemie plików (GCS)
* Interaktywna analiza rozproszonych danych przy wykorzystaniu modułu Spark Core i Spark SQL
* Wprowadzenie modułu pandas
* Wizualizacje przy wykorzystaniu matplotlib oraz seaborn

Języki:
* bash, Python, SQL 

Krótka lista przydatnych poleceń bash: https://www.reddit.com/r/linux/comments/9rns12/some_linux_commands_cheatsheet/

### Jupyter Notebook

* Interaktywny notatnik dostepny poprzez przeglądarkę.
* Służy do wpisywania poleceń w wybranych jezykach programowania oraz opis w tzw jezyku markdown.
* Polecenia/opis wpisujemy do komórek. Komórki są wykonywane przez tzw. kernel (np Python w określonej wersji)
* Działa w tryb edycji komórek + tryb wykonywania poleceń. (przejście poprzez ESC i ENTER)
* Podstawowe skróty klawiaturowe: 
  * Esc/Enter
  * strzalki
  * ctrl-enter/shift-enter
  * a/b/d
  * y/m 
* Moze wykonywać polecenia powłoki (bash) !  %%bash
* Notatnik zapisywany jest w formacie ipynb (IPythonNoteBook)
* Kolejnosc uruchomienia komórek moze byc rozna.
* Mozliwosc zatrzymania kernela i uruchomienia od nowa. 

Polecamy film: https://www.youtube.com/watch?v=HW29067qVWk

Przykładowe komórki Pythonowe i Bashowe:

In [1]:
import os
user = os.environ.get('USER')
msg = f"Hello {user}!"
print (msg)

Hello tgambin!


In [2]:
! pwd # pokaż aktualną scieżkę

/home/jovyan/work/git/ds-notebooks/session_1


In [3]:
%%bash
head 01_intro.ipynb
echo "--------------------"
tail 01_intro.ipynb

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Zajęcia 1 (część 1)\n",
    "\n",
    "Zakres:\n",
    "* Zapoznanie sie ze srodowiskiem pracy (GKE na GCP)\n",
--------------------
   "version": "3.7.12"
  },
  "notebook_test": {
   "keytab_path": "/data/work/home/ds-lab-testuser1/ds-lab-testuser1.keytab",
   "user": "ds-lab-testuser1"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


## Kopiowanie danych do swojego katalogu domowego

W katalogu `/mnt/data/datascience` znajduje sie plik który bedzie nam dzisiaj słuzyl do pracy. Należy go skopiować do swojego katalogu domowego. 
Dane pochodzą z https://gdac.broadinstitute.org/ i zawierają dane z badaniami nad mutacjami typu CNV w obrebie genu BRAC2.


In [ ]:
%%bash
mkdir -p ~/data     # stworz katalog data (i posrednie)
cp /mnt/data/datascience/brca.txt ~/data  # skopiuj plik z lokalizacji do bieżącej lokalizacji
ls -lah   ~/data       # wylistuj zawartość bieżącego katalogu

In [ ]:
%%bash
head ~/data/brca.txt   # pokaż początek pliku
echo                   # pusta linia
tail ~/data/brca.txt   # pokaż koniec pliku 
echo
wc -l ~/data/brca.txt  # policz linie pliku

# Środowisko Google Kubernetes Engine 

Logowanie kontem github. 


## Kubernetes

Kubernetes (k8s) to otwarta platforma do koordynacji wysoko dostępnego klastra. 
* Umożliwia  deklaratywną konfigurację, automatyzację wdrażania, skalowanie i autoskalowanie rozwiązań.
* Pozwala uruchamiać aplikacje/narzędzie bez przypisywania ich do konkretnej maszyny. Aplikacje muszą być niezależne od konkretnego serwera: muszą być skonteneryzowane.
* Możliwe do uruchomienia na prywatnym centrum danych, infrastrukturze hybrydowej lub chmurze publicznej.


**Klaster Kubernetes**  
* **Węzeł sterujący (Master node)** koordynuje działanie klastra np. zlecanie uruchomienia aplikacji, utrzymywanie pożądanego stanu aplikacji, skalowanie aplikacji i instalowanie nowych wersji
* **Węzeł roboczy (Worker node)** Na węzłach uruchamiane są aplikacje. Na każdym węźle działa agent zarządzający tym węzłem i komunikujący się z masterem Kubernetes (API). Węzeł zawiera także narzędzia do obsługi kontenerów

**Pod** to grupa jednego/wielu kontenerów wraz ze wspólnymi zasobami (np. dysk). Pod tworzy "wirtualney serwer" i może zawierać różne kontenery aplikacji współdzielące zasoby i kontekst wykonawczy na tym samym węźle.
* Pod jest uruchamiany na węźle roboczym. Węzeł jest maszyną roboczą, fizyczną lub wirtualną, w zależności od klastra. 
* Węzeł może zawierać wiele podów. 
* Kubernetes master automatycznie zleca uruchomienie podów na różnych węzłach w ramach klastra. Automatyczne zlecanie uruchomienia bierze pod uwagę zasoby dostępne na każdym z węzłów.

![](../img/gke.png) 


## Wykorzystanie Google Cloud Storage

#### Projekt
* Wszystki dane przynależą do konkretnego projektu.
* Do projektu mogą mieć dostęp użytkownicy. 
* Projekt ma zdefiniowane metody uwierzytelniające, rozliczenia, monitorowanie etc. 
#### Kubełek (bucket)
* Kubełek (buckket) to kontener na pliki/obiekty. 
* Nazwa Bucketu musi być unikalna w skali całej usługi u wszystkich użytkowników (!) 
* Kubełków nie można zagnieżdzać
* W kubełkach możemy tworzyć foldery i tam logicznie grupować pliki.
* Kubełek wraz z zawartością może zostać udostępniony publicznie.
* Kubełkowi nie można zmienić nazwy lub metadanych. Trzeba go usunąć i stworzyć ponownie.
#### Obiekt 
* obiekty przechowywane w kubełkach 
* obiekty mają zawartość oraz metadane
* obiekty są niemodyfikowalne 

Do operacji na Google Storage można wykorzystać narzędzie `gsutil`: 

##### Operacje na kubełkach
* listowanie kubełków (buckets) - `ls`
* tworzenie nowego kubełka - `mb`
* usuwania kubełka - `rm`
* listowanie zawartości kubełków - `ls` 
* udostępnianie - `iam`

##### Operacje na obiektach
* dodawania pliku do kubełka - `cp`
* kopiowanie - `cp`
* usuniecie z kubełka - `cp`
* pobranie informacji o obiekcie - `stat`

#### Operacje na kubełkach

In [4]:
! gsutil ls

gs://bdg-lab-agaszmurlo/
gs://bdg-lab-kkulpa7/
gs://bdg-lab-tgambin/
gs://bdg-lab-tgambin1/
gs://bigdata-datascience/
gs://bucket-agaszmurlo-1/
gs://bucket-kkulpa7/
gs://lab-ds-infra/


In [5]:
! echo $USER  # wyswietlenie nazwy uzytkownika

tgambin


In [6]:
! gsutil mb gs://bdg-lab-$USER  # stworzenie bucketu

Creating gs://bdg-lab-tgambin/...
ServiceException: 409 Bucket bdg-lab-tgambin already exists.


In [7]:
! gsutil ls gs://bdg-lab-$USER 

gs://bdg-lab-tgambin/survey_results_public.csv
gs://bdg-lab-tgambin/survey/


In [27]:
#! gsutil cp brca.txt gs://bdg-lab-$USER/

Copying file://brca.txt [Content-Type=text/plain]...
/ [1 files][366.8 KiB/366.8 KiB]                                                
Operation completed over 1 objects/366.8 KiB.                                    


In [8]:
! gsutil du -s  gs://bdg-lab-$USER  # ile zajmuje przestrzeni?

189207776    gs://bdg-lab-tgambin


In [9]:
! gsutil ls -L -b gs://bdg-lab-$USER  # listowanie zawartości 

gs://bdg-lab-tgambin/ :
	Storage class:			STANDARD
	Location type:			multi-region
	Location constraint:		US
	Versioning enabled:		None
	Logging configuration:		None
	Website configuration:		None
	CORS configuration: 		None
	Lifecycle configuration:	None
	Requester Pays enabled:		None
	Labels:				None
	Default KMS key:		None
	Time created:			Wed, 15 Dec 2021 21:14:58 GMT
	Time updated:			Wed, 15 Dec 2021 21:14:58 GMT
	Metageneration:			1
	Bucket Policy Only enabled:	False
	ACL:				
	  [
	    {
	      "entity": "project-owners-9940977455",
	      "projectTeam": {
	        "projectNumber": "9940977455",
	        "team": "owners"
	      },
	      "role": "OWNER"
	    },
	    {
	      "entity": "project-editors-9940977455",
	      "projectTeam": {
	        "projectNumber": "9940977455",
	        "team": "editors"
	      },
	      "role": "OWNER"
	    },
	    {
	      "entity": "project-viewers-9940977455",
	      "projectTeam": {
	        "projectNumber": "9940977455",
	        "team": "vie

#### Operacje na zawartości kubełków

In [10]:
! gsutil ls -r gs://bdg-lab-$USER  # listowanie zawartosci kubełka

gs://bdg-lab-tgambin/survey_results_public.csv
gs://bdg-lab-tgambin/survey/:

gs://bdg-lab-tgambin/survey/2020/:
gs://bdg-lab-tgambin/survey/2020/survey_results_public.csv


In [11]:
! gsutil cp ~/work/git/ds-notebooks/README.md gs://bdg-lab-$USER  # upload obiektu do kubełka

Copying file:///home/jovyan/work/git/ds-notebooks/README.md [Content-Type=text/markdown]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


In [12]:
! gsutil ls -r gs://bdg-lab-$USER # listowanie zawartości

gs://bdg-lab-tgambin/README.md
gs://bdg-lab-tgambin/survey_results_public.csv
gs://bdg-lab-tgambin/survey/:

gs://bdg-lab-tgambin/survey/2020/:
gs://bdg-lab-tgambin/survey/2020/survey_results_public.csv


In [13]:
! gsutil iam get gs://bdg-lab-$USER

{
  "bindings": [
    {
      "members": [
        "projectEditor:bigdata-datascience",
        "projectOwner:bigdata-datascience"
      ],
      "role": "roles/storage.legacyBucketOwner"
    },
    {
      "members": [
        "projectViewer:bigdata-datascience"
      ],
      "role": "roles/storage.legacyBucketReader"
    }
  ],
  "etag": "CAE="
}


In [14]:
! gsutil stat gs://bdg-lab-$USER/README.md # metadane obiektu w kubełku

gs://bdg-lab-tgambin/README.md:
    Creation time:          Sat, 18 Dec 2021 10:29:10 GMT
    Update time:            Sat, 18 Dec 2021 10:29:10 GMT
    Storage class:          STANDARD
    Content-Language:       en
    Content-Length:         14
    Content-Type:           text/markdown
    Hash (crc32c):          e+o8YQ==
    Hash (md5):             MxzbFtnEwBhVlFmJHMLZeQ==
    ETag:                   CIjMl4KS7fQCEAE=
    Generation:             1639823350031880
    Metageneration:         1


In [ ]:
! gsutil iam ch allUsers:objectViewer gs://bdg-lab-$USER # dodanie uprawnien do odczytu

Po wykonaniu tego polecenia nasz kubełek staje się publiczny i możemy się do niego 
http://storage.googleapis.com/bucket-NAZWA_UZYTKOWNIKA. Zawartość pliku można odczytać poprzez http://storage.googleapis.com/bucket-NAZWA_UZYTKOWNIKA/NAZWA_OBIEKTU

In [ ]:
! gsutil iam ch -d allUsers:objectViewer gs://bdg-lab-$USER # usuniecie uprawnien do odczytu

Teraz można ponownie zweryfikowac możliwość publicznego odczytu danych z kubełka.

#### Przeniesienie do docelowego kubełka

## <span style='background:yellow'> ZADANIE 1 </span>
Wgraj plik brca.txt do kubełka `gs://bdg-lab-$USER`.

## <span style='background:yellow'> ZADANIE 2 </span>
a) Utwórz plik zawierający aktualną date i godzinę. 
b) Udostępnij plik publicznie na google storage. 

### Pobranie pliku z powrotem na lokalny dysk

In [ ]:
! gsutil cp gs://bdg-lab-$USER/brca.txt .  # gdzie sie zapisal ten plik?

## Apache Spark

* Apache Spark platforma ogólnego zastosowania, opensource, do przetwarzania duzych zbiorow danych.
* Posiada  API dla języków programowania: Scala, Python i R. 
* Przetwarzanie w Spark przetwarzanie jest wykonywane w większości  wprost w pamięci operacyjnej.
* Przeznaczenie: do uruchamiania  aplikacji i skryptów z wykorzystaniem uczenia maszynowego lub interaktywnych kwerend.
* Spark ten wspiera SQL (typ DataFrames), przetwarzanie strumieniowe oraz przetwarzanie grafów.
* Integracja z lokalną pamięci masową, rozproszonymi lub obiektowymi systemu plików.
* Spark można uruchamić na pojedynczej maszynie na środowisku klastrowym, lub w chmurze. 

Zakres na dzisiejsze laboratorium:
* stworzenie sesji Spark
* zaczytanie danych z pliku tekstowego
* kwerendy na danych

Bedziemy korzystac z pliku ktory zapisalismy na GCS w pierwszej czesci cwiczenia.

In [15]:
! gsutil ls -r gs://bdg-lab-$USER/

gs://bdg-lab-tgambin/README.md
gs://bdg-lab-tgambin/survey_results_public.csv
gs://bdg-lab-tgambin/survey/:

gs://bdg-lab-tgambin/survey/2020/:
gs://bdg-lab-tgambin/survey/2020/survey_results_public.csv


### Przygotowanie sesji Sparkowej


In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.executor.instances", "1") \
.config("spark.executor.memory", "1g") \
.getOrCreate()

In [ ]:
spark.stop()

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.executor.instances", "1") \
.config("spark.executor.memory", "1g") \
.getOrCreate()

### Ręczne utworzenie zbioru danych

In [18]:
# stworzenie rozproszonego DF na podstawie podanej listy krotek 
df_users = spark.createDataFrame(
    [(123, 'Jan Kowalski', 'jan@kowalski.pl'), 
     (124, 'Anna Nowak', 'anna@nowak.pl'), 
     (125, 'Janusz Kowalski', 'janusz@kowalski.pl'), 
     (126, 'Anita Nowak', 'anita@nowak.pl')])

In [19]:
df_users.show()

+---+---------------+------------------+
| _1|             _2|                _3|
+---+---------------+------------------+
|123|   Jan Kowalski|   jan@kowalski.pl|
|124|     Anna Nowak|     anna@nowak.pl|
|125|Janusz Kowalski|janusz@kowalski.pl|
|126|    Anita Nowak|    anita@nowak.pl|
+---+---------------+------------------+



In [20]:
df_users.printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)



In [21]:
df_users = spark.createDataFrame(
    [(123, 'Jan Kowalski', 'jan@kowalski.pl'), 
     (124, 'Anna Nowak', 'anna@nowak.pl'), 
     (125, 'Janusz Kowalski', 'janusz@kowalski.pl'), 
     (126, 'Anita Nowak', 'anita@nowak.pl')],
    schema='id int not null, name string, email string')

In [22]:
df_users.show()

+---+---------------+------------------+
| id|           name|             email|
+---+---------------+------------------+
|123|   Jan Kowalski|   jan@kowalski.pl|
|124|     Anna Nowak|     anna@nowak.pl|
|125|Janusz Kowalski|janusz@kowalski.pl|
|126|    Anita Nowak|    anita@nowak.pl|
+---+---------------+------------------+



In [23]:
df_users.printSchema()

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)




### Wczytanie danych
Zazwyczaj dane mamy przechowane na zewnątrz naszego notatnika (np. w pliku) i należy je odczytać.

In [24]:
import os
user_name = os.environ.get('USER')
print(user_name)

tgambin


In [25]:
input_path = f'gs://bdg-lab-{user_name}/brca.txt'

In [28]:
df = spark.read.load(input_path, format="csv", sep="\t", inferSchema="true", header="true")

### Charakterystyka danych

In [29]:
type (df)                 # jaki jest typ danych

pyspark.sql.dataframe.DataFrame

In [30]:
df.explain()              # fizyczny plan wykonania

== Physical Plan ==
FileScan csv [Sample#54,Chromosome#55,Start#56,End#57,Num_Probes#58,Segment_Mean#59] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[gs://bdg-lab-tgambin/brca.txt], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Sample:string,Chromosome:int,Start:int,End:int,Num_Probes:string,Segment_Mean:double>




In [31]:
df.explain(True)          # logiczny i fizyczny plan wykonania

== Parsed Logical Plan ==
Relation[Sample#54,Chromosome#55,Start#56,End#57,Num_Probes#58,Segment_Mean#59] csv

== Analyzed Logical Plan ==
Sample: string, Chromosome: int, Start: int, End: int, Num_Probes: string, Segment_Mean: double
Relation[Sample#54,Chromosome#55,Start#56,End#57,Num_Probes#58,Segment_Mean#59] csv

== Optimized Logical Plan ==
Relation[Sample#54,Chromosome#55,Start#56,End#57,Num_Probes#58,Segment_Mean#59] csv

== Physical Plan ==
FileScan csv [Sample#54,Chromosome#55,Start#56,End#57,Num_Probes#58,Segment_Mean#59] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[gs://bdg-lab-tgambin/brca.txt], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Sample:string,Chromosome:int,Start:int,End:int,Num_Probes:string,Segment_Mean:double>



In [32]:
df.rdd.getNumPartitions() # liczba partycji (bloków danych)

1

In [33]:
df.printSchema()          # schemat danych

root
 |-- Sample: string (nullable = true)
 |-- Chromosome: integer (nullable = true)
 |-- Start: integer (nullable = true)
 |-- End: integer (nullable = true)
 |-- Num_Probes: string (nullable = true)
 |-- Segment_Mean: double (nullable = true)



In [34]:
df.count()                # wymiary (liczba wierszy)

5298

In [35]:
len(df.columns)           # wymiary (liczba kolumn)

6

In [36]:
df.describe().show()      # pokaż podsumowanie danych w tabeli

+-------+--------------------+------------------+-------------------+-------------------+----------+------------------+
|summary|              Sample|        Chromosome|              Start|                End|Num_Probes|      Segment_Mean|
+-------+--------------------+------------------+-------------------+-------------------+----------+------------------+
|  count|                5298|              5298|               5298|               5298|      5298|              5298|
|   mean|                null|11.312948282370705|5.464405806266516E7|7.557261170932427E7|      null|0.2103169573704153|
| stddev|                null| 6.548797844563064|5.276515600857485E7|5.816178219098644E7|      null|0.8318029444477086|
|    min|TCGA-A2-A0EU-01A-...|                 1|                  1|              99263|        NA| -3.49124544908987|
|    max|TCGA-BH-A0W5-10A-...|                24|          249152065|          249240606|        NA|  4.25258455583762|
+-------+--------------------+----------

In [37]:
df.describe("Chromosome").show() # pokaż podsumowanie danych w kolumnie 

+-------+------------------+
|summary|        Chromosome|
+-------+------------------+
|  count|              5298|
|   mean|11.312948282370705|
| stddev| 6.548797844563064|
|    min|                 1|
|    max|                24|
+-------+------------------+



### Odczyt danych
Z wykorzystaniem API DataFrame

In [38]:
df.show()                    # pokaż 20 wierszy danych 

+--------------------+----------+---------+---------+----------+-------------------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|
+--------------------+----------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|
|TCGA-A2-A0EU-10A-...|         5|    11770|180905246|        NA|  0.176064461471459|
|TCGA-A2-A0EU-10A-...|         6|    63815|171050932|        NA|-

In [39]:
df.show(10, truncate=False) # pokaż 10 wierszy, nie skracaj danych

+----------------------------+----------+---------+---------+----------+-------------------+
|Sample                      |Chromosome|Start    |End      |Num_Probes|Segment_Mean       |
+----------------------------+----------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-01D-A060-02|1         |10209    |2583075  |NA        |-0.210121164888764 |
|TCGA-A2-A0EU-10A-01D-A060-02|1         |2583076  |249240606|NA        |-0.101789555120838 |
|TCGA-A2-A0EU-10A-01D-A060-02|2         |10002    |243189359|NA        |-0.337963741969504 |
|TCGA-A2-A0EU-10A-01D-A060-02|3         |60175    |162511435|NA        |-0.0850647472883194|
|TCGA-A2-A0EU-10A-01D-A060-02|3         |162511436|162626067|NA        |0.42309157125509   |
|TCGA-A2-A0EU-10A-01D-A060-02|3         |162626068|197962416|NA        |-0.0962225847981023|
|TCGA-A2-A0EU-10A-01D-A060-02|4         |69223    |191014397|NA        |-0.0713350286479217|
|TCGA-A2-A0EU-10A-01D-A060-02|5         |11770    |180905246|NA       

In [ ]:
df.select("Sample").show()  # pokaż tylko kolumne sample

In [ ]:
df.select("Chromosome", "Start", "End").show() # pokaż kolumny chromosome, start i end

In [ ]:
df_chrom = df.select(df.Chromosome).distinct() # stworz nowy DF zawierajacy tylko unikalne wartosci chromosomow

In [ ]:
df_chrom.count()                     # policz wiersze w nowym DF

In [ ]:
df.filter("Chromosome > 21").show() # pokaz dane spelniajace warunek (przy uzyciu filter)

In [ ]:
df.where("Chromosome > 21").show() # pokaz dane spelniajace warunek (przy uzyciu when)

In [ ]:
df.filter("Chromosome > 21").explain()  # pokaz plan wykonania

In [ ]:
df.filter("Chromosome > 21 and Segment_Mean > 0").show() # pokaz dane spelniajace warunki

### Grupowanie i funkcje agregujące

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df.groupBy("Chromosome").count().show()   # dokonaj grupowania po chromosomie i policz rekordy w grupie

In [ ]:
df.groupBy("Chromosome").avg("Segment_Mean").show() # dokonaj grupowania po chromosomie i policz srednia wartosc segment_mean w grupie

In [ ]:
# pogrupuj dane wzgledem probki i chromosomu, policz rekordy w grupie
df.groupBy("Sample","Chromosome").count().orderBy(asc("Sample")).show()

### Kolumny wyliczane

In [ ]:
df.withColumn ("Length", col("End") - col("Start")).show() # dodaj kolumne dlugosc jako end - start

In [ ]:
df.withColumn ("Material", lit("DNA")).show()     # dodaj kolumne o stalej wartosci 'DNA'

In [ ]:
df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome"))).show() # dodaj kolumne z konkatencja

In [ ]:
df.drop("Chromosome").show()  # usun kolumne Chromosome. 
# Czy DF została pozbawiona kolumny na trwale?

### Zapis wyników

In [ ]:
df2 = df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome")))  # dodaj kolumne z konkatenacja i zapisz do nowego DF

In [ ]:
output_path = f'gs://bdg-lab-{user_name}/brca2.txt'
df2.write.format("csv").mode("overwrite").save(output_path)

In [ ]:
! gsutil ls gs://bdg-lab-$USER

In [ ]:
!gsutil cat gs://bdg-lab-$USER/brca.txt | head

In [40]:
spark.stop()

## <span style='background:yellow'> ZADANIE 3</span>

a) Ile jest unikalnych danych próbek w tym zbiorze danych?

b) Pokaż nazwę próbki, numer chromosomu, początek i koniec segmentu dla segmentów występujących na chromosomie 21,22,23 i mających startową pozycję większą niż 10000000. 

c) Ile występuje wierszy dla każdej z próbki? Pokaż wynik z pełną nazwą próbki. Zapisz wynik do pliku na GCS